<a href="https://colab.research.google.com/github/elenancalima/Troph_Min_5to2/blob/main/Minimal_5_to_2_backbone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === Setup (mount + deps + project path) — weights-only, no datasets ===
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Pin a stable Ultralytics that doesn't get “creative” on import,
# and install imageio for PNG writing.
# Using version 8.3.0 to match the yolo11n.pt weights release.
%pip -q install -U "ultralytics==8.3.0" "imageio>=2.31.0"

import os, sys, shutil, pathlib, urllib.request, importlib.metadata as im

# Work under Drive so artifacts persist
PROJ_DIR = "/content/drive/MyDrive/Troph_Min_5to_2_mod"
os.makedirs(PROJ_DIR, exist_ok=True)
os.chdir(PROJ_DIR)
if PROJ_DIR not in sys.path:
    sys.path.insert(0, PROJ_DIR)

print("Project dir:", PROJ_DIR)

# Sanity printouts (no ultralytics import here to avoid side-effects)
import torch
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
try:
    print("Ultralytics:", im.version("ultralytics"))
except Exception as _e:
    print("Ultralytics: not found")

# --- Weights-only bootstrap (no YOLO() call here) ---
YOLO_WEIGHTS_NAME = "yolo11n.pt"
YOLO_WEIGHTS_PATH = os.path.join(PROJ_DIR, YOLO_WEIGHTS_NAME)
if not os.path.isfile(YOLO_WEIGHTS_PATH):
    url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt"
    print(f"[weights] Downloading {YOLO_WEIGHTS_NAME}…")
    urllib.request.urlretrieve(url, YOLO_WEIGHTS_PATH)
    print(f"[weights] Ready at: {YOLO_WEIGHTS_PATH}")
else:
    print(f"[weights] Found: {YOLO_WEIGHTS_PATH}")

# Quiet standard logging from the 'ultralytics' logger (prevents banner spam if it appears)
import logging
logging.getLogger("ultralytics").setLevel(logging.ERROR)

# imageio v3 check
from imageio import v3 as iio
print("imageio v3 import: OK")

Mounted at /content/drive
Project dir: /content/drive/MyDrive/Troph_Min_5to_2_mod
Torch: 2.8.0+cu126 | CUDA available: True
Ultralytics: 8.3.0
[weights] Found: /content/drive/MyDrive/Troph_Min_5to_2_mod/yolo11n.pt
imageio v3 import: OK


In [2]:
#@title Fast local dev: mirror code from Drive → /content and import from there

import os, sys, subprocess, time

# Paths
PROJ_DIR_DRIVE = "/content/drive/MyDrive/Troph_Min_5to_2_mod"   # source of truth on Drive
PROJ_DIR_LOCAL = "/content/Troph_Min_5to_2_mod"                  # fast local mirror for imports
WEIGHTS_DIR    = f"{PROJ_DIR_DRIVE}/weights"                     # keep weights on Drive (persistent)

# Drive is already mounted in Cell 1; avoid remounting to prevent race with rsync.
# Briefly wait until the Drive folder is actually visible.
for _ in range(20):
    if os.path.isdir(PROJ_DIR_DRIVE):
        break
    time.sleep(0.25)

os.makedirs(PROJ_DIR_DRIVE, exist_ok=True)
os.makedirs(PROJ_DIR_LOCAL, exist_ok=True)

# Mirror Drive → local (exclude caches/heavy run dirs)
cmd = [
    "rsync","-ah","--delete","--info=stats2",
    "--exclude","__pycache__/",
    "--exclude",".ipynb_checkpoints/",
    "--exclude","weights/",
    "--exclude","runs/",
    PROJ_DIR_DRIVE + "/", PROJ_DIR_LOCAL + "/"
]
proc = subprocess.run(cmd, capture_output=True, text=True)
if proc.returncode != 0:
    # Show rsync diagnostics, then raise the same error Colab would have raised
    print(proc.stdout)
    print(proc.stderr)
    raise subprocess.CalledProcessError(proc.returncode, cmd)

# Import from local
if PROJ_DIR_LOCAL not in sys.path:
    sys.path.insert(0, PROJ_DIR_LOCAL)

print("CODE FROM:", PROJ_DIR_LOCAL)
print("WEIGHTS  :", WEIGHTS_DIR)


CODE FROM: /content/Troph_Min_5to_2_mod
WEIGHTS  : /content/drive/MyDrive/Troph_Min_5to_2_mod/weights


In [3]:
#@title Import modules from local mirror
import importlib

import config;          importlib.reload(config)
import fake_data;       importlib.reload(fake_data)
import folder_dataset;  importlib.reload(folder_dataset)
import data_factory;    importlib.reload(data_factory)
import model_factory;   importlib.reload(model_factory)
import trainer;         importlib.reload(trainer)
import pipeline;        importlib.reload(pipeline)
import troph;           importlib.reload(troph)

from troph import Troph
print("Imports ready (local).")


Imports ready (local).


## Config cell

In [4]:
# === Config (control panel) ===
# What to run
DO_FAKE_GEN            = False   # generate fake train/val folders on disk
DO_FAKE_TRAIN_INFER    = False   # train+infer on the fake folders
DO_FIELD_TRAIN_INFER   = True    # train+infer on field data from .tar files
DO_BATCH_PROCESS       = False   # (reserved) batch predict over many roots

# Paths (edit as needed)
# Fake (local, fast)
FAKE_TRAIN_ROOT = "/content/fake_train_v1"
FAKE_VAL_ROOT   = "/content/fake_val_v1"

# Field data (Drive .tar archives → extracted to local)
TRAIN_TAR = "/content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot3_simp.tar"  # <- EDIT
VAL_TAR   = "/content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot4_simp.tar"  # <- EDIT

# Where to extract those .tars locally
LOCAL_TRAIN_ROOT = "/content/_field_local/train"
LOCAL_VAL_ROOT   = "/content/_field_local/val"

# Weights (persist on Drive)
WEIGHTS_DIR = "/content/drive/MyDrive/Troph_Min_5to_2_mod/weights"

# Inference temp root (always write locally first, then sync to Drive)
INFER_TMP_ROOT = "/content/_troph_tmp_preds"   # ← NEW

# Training & inference knobs
EPOCHS          = 10
BATCH_SIZE      = 8
NUM_WORKERS     = 0           # 0 is safer with Drive-backed data; local is fine too
PROGRESS_EVERY  = 50

# Temporal context (TCN-ready; keep 1 until temporal trainer is wired)
SEQ_LEN = 1                   # must be odd when >1 (e.g., 3,5,7,...); anchor is the center frame

# Label dilation (loss pre-processing)
LABEL_DILATE_RADIUS = 4       # px at 170×170 resolution; used in loss label preprocessing

# Output & thresholds (for inference binarization)
THRESH_TROPH = 0.5
THRESH_PART  = 0.5

# YOLO backbone (feature encoder for Stage 1)
YOLO_BACKBONE_VARIANT = "yolo11n"   # informational; we load weights from the path below
YOLO_BACKBONE_WEIGHTS = "/content/drive/MyDrive/_pretrained/yolo11n.pt"  # COCO-pretrained
BACKBONE_FREEZE_EPOCHS = 5          # keep backbone frozen for first N epochs, then unfreeze
YOLO_FEATURE_STRIDE    = 5          # we need 850→170; stride 5 aligns features to 170×170

print("[config] fake_gen=", DO_FAKE_GEN,
      "| fake_train_infer=", DO_FAKE_TRAIN_INFER,
      "| field_train_infer=", DO_FIELD_TRAIN_INFER,
      "| batch=", DO_BATCH_PROCESS)
print("[config] TRAIN_TAR:", TRAIN_TAR)
print("[config] VAL_TAR  :", VAL_TAR)
print("[config] WEIGHTS  :", WEIGHTS_DIR)
print("[config] INFER_TMP_ROOT:", INFER_TMP_ROOT)
print("[config] SEQ_LEN:", SEQ_LEN, "| EPOCHS:", EPOCHS, "| FREEZE:", BACKBONE_FREEZE_EPOCHS)


[config] fake_gen= False | fake_train_infer= False | field_train_infer= True | batch= False
[config] TRAIN_TAR: /content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot3_simp.tar
[config] VAL_TAR  : /content/drive/MyDrive/MainConnection_VidRoots/tmpvidroot4_simp.tar
[config] WEIGHTS  : /content/drive/MyDrive/Troph_Min_5to_2_mod/weights
[config] INFER_TMP_ROOT: /content/_troph_tmp_preds
[config] SEQ_LEN: 1 | EPOCHS: 10 | FREEZE: 5


In [5]:
# === TAR staging (assume single top folder, no conditionals) ===
import os, shutil, subprocess

def stage_tar_to_root(tar_path: str, dest_root: str):
    """Extract `tar_path` into `dest_root`, assuming the tar was created from a single
    top folder that contains the expected subfolders. We always:
      • wipe dest_root
      • extract to a temp dir
      • move the single top folder's contents into dest_root
    No structure probing; let errors surface if the tar is malformed.
    """
    tmp = dest_root + "__extract"

    # clean
    if os.path.isdir(dest_root):
        shutil.rmtree(dest_root, ignore_errors=True)
    if os.path.isdir(tmp):
        shutil.rmtree(tmp, ignore_errors=True)
    os.makedirs(tmp, exist_ok=True)

    # extract into tmp
    subprocess.run(["tar", "-xf", tar_path, "-C", tmp], check=True)

    # move contents of the single top folder into dest_root
    top = os.path.join(tmp, os.listdir(tmp)[0])  # assume exactly one entry
    os.makedirs(dest_root, exist_ok=True)
    for name in os.listdir(top):
        shutil.move(os.path.join(top, name), os.path.join(dest_root, name))

    shutil.rmtree(tmp, ignore_errors=True)
    print(f"[stage] {os.path.basename(tar_path)} -> {dest_root}")


In [6]:
# === Priors: require precomputed (no creation) ===
import os
from config import (
    SUB_RGB_850, SUB_AB_MASK_850, SUB_FB_MASK_850,
    SUB_SIMPLE_POINT_170, SUB_SIMPLE_LINE_170,
    SUB_LABEL_POINT_170, SUB_LABEL_LINE_170,
)

def require_priors_exist(root: str):
    pt = os.path.join(root, SUB_SIMPLE_POINT_170)
    ln = os.path.join(root, SUB_SIMPLE_LINE_170)
    if not os.path.isdir(pt) or not os.listdir(pt):
        raise FileNotFoundError(f"[priors] missing or empty: {pt}")
    if not os.path.isdir(ln) or not os.listdir(ln):
        raise FileNotFoundError(f"[priors] missing or empty: {ln}")
    print(f"[priors] OK at {root}")

def require_field_inputs_exist(root: str, need_labels: bool):
    req = [SUB_RGB_850, SUB_AB_MASK_850, SUB_FB_MASK_850,
           SUB_SIMPLE_POINT_170, SUB_SIMPLE_LINE_170]
    if need_labels:
        req += [SUB_LABEL_POINT_170, SUB_LABEL_LINE_170]
    missing = [d for d in req if not os.path.isdir(os.path.join(root, d))]
    if missing:
        raise FileNotFoundError(f"[field] missing subfolders under {root}: {missing}")


In [7]:
# === Orchestrator · Step-1 (YOLO-TCN) ===
# Always writes inference to local tmp first, then syncs to Drive.

import importlib
import troph;     importlib.reload(troph)
from troph import Troph
import fake_data; importlib.reload(fake_data)

def _train_and_predict(model: Troph, train_root: str, val_root: str):
    # ---- Train ----
    model.train(
        train_root=train_root,
        val_root=val_root,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        seq_len=SEQ_LEN,
        label_dilate_radius=LABEL_DILATE_RADIUS,
        backbone_freeze_epochs=BACKBONE_FREEZE_EPOCHS,  # ← consistent with config
    )
    model.save(WEIGHTS_DIR)

    # ---- Infer (local tmp → then copy back) ----
    summary = model.predict(
        root=val_root,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        clear_outputs=True,
        progress_every=PROGRESS_EVERY,
        seq_len=SEQ_LEN,
        tmp_root=INFER_TMP_ROOT,  # ← local-first writing handled inside troph.predict
    )
    print(summary)

# — Run by toggles from the Config cell —
if DO_FAKE_GEN:
    print("[run] Generating FAKE train/val on disk…")
    fake_data.make_fake_trainval_folders(
        FAKE_TRAIN_ROOT, FAKE_VAL_ROOT, n_train=256, n_val=64, seed=0
    )
    print("[run] Fake train/val ready.")

if DO_FAKE_TRAIN_INFER:
    print("[run] Fake train/infer (sanity check)…")
    model = Troph()  # fresh
    _train_and_predict(model, FAKE_TRAIN_ROOT, FAKE_VAL_ROOT)

if DO_FIELD_TRAIN_INFER:
    print("[run] Field train/infer from .tar (no priors generation)")
    stage_tar_to_root(TRAIN_TAR, LOCAL_TRAIN_ROOT)
    stage_tar_to_root(VAL_TAR,   LOCAL_VAL_ROOT)

    require_field_inputs_exist(LOCAL_TRAIN_ROOT, need_labels=True)
    require_field_inputs_exist(LOCAL_VAL_ROOT,   need_labels=True)
    require_priors_exist(LOCAL_TRAIN_ROOT)
    require_priors_exist(LOCAL_VAL_ROOT)

    model = Troph()  # fresh by default
    _train_and_predict(model, LOCAL_TRAIN_ROOT, LOCAL_VAL_ROOT)

if DO_BATCH_PROCESS:
    print("[run] Batch processing reserved/off here.")


[run] Field train/infer from .tar (no priors generation)
[stage] tmpvidroot3_simp.tar -> /content/_field_local/train
[stage] tmpvidroot4_simp.tar -> /content/_field_local/val
[priors] OK at /content/_field_local/train
[priors] OK at /content/_field_local/val
New https://pypi.org/project/ultralytics/8.3.218 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det

RuntimeError: Dataset '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' error ❌ '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' does not exist

Multiple datasets

Report training loss too

Data output test (visually and numerically) tar export

The tag consistency problem

Redesign loss/targets for sparse labels

Make it so it only kills simple pred masks? ...

Resume from here

Resume from here.

Now, dummy training.


BELOW SETUP IS UNTESTED

ACTUAL RUN STARTS HERE

This takes considerably longer than YOLO inference, beware.